In [1]:
%fs ls


In [2]:
%fs ls /FileStore/tables/

In [3]:
jsonData = spark.read.json("/FileStore/tables/yelp_training_set_review.json")

In [4]:
display(jsonData)

In [5]:
#yelp_class = jsonData[(jsonData['stars'] == 1) | (jsonData['stars'] == 5)]

In [6]:
jsonData.show(5)

In [7]:
jsonData.select("text").first()

In [8]:
data1=jsonData.select("text","stars")

In [9]:
display(data1)

In [10]:
data1.count()

In [11]:
from pyspark.ml.feature import StopWordsRemover


In [12]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [13]:
tokenizer = RegexTokenizer(inputCol="text", outputCol="words")
tokenized = tokenizer.transform(data1)

In [14]:
display(tokenized)

In [15]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
data3=remover.transform(tokenized)

In [16]:
display(data3)

In [17]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import NGram

In [18]:
cv = CountVectorizer(inputCol="filtered", outputCol="Count",minDF=4.0)
ngram = NGram(n=2, inputCol="filtered", outputCol="ngrams")

In [19]:
model = cv.fit(data3)
#ng=ngram.fit(data3)
#from pyspark.ml.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import udf,col
mergeCols = udf(lambda fruits, meat: fruits + meat)
def changes(data):
  return data.withColumn("Features", mergeCols(col("filtered"), col("ngrams")))

In [20]:
data4=model.transform(ngram.transform(data3))
data4.dtypes
#assembler = VectorAssembler(
#    inputCols=["ngrams", "filtered"],
#    outputCol="Features")
#output = assembler.transform(data4)
#output.show()

In [21]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures")
#featurizedData = hashingTF.transform(wordsData)
idf = IDF(inputCol="rawFeatures", outputCol="TFIDF")
#idfModel = idf.fit(featurizedData)
#rescaledData = idfModel.transform(featurizedData)

In [22]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[tokenizer, remover,cv,ngram,hashingTF,idf])
pipelinefit=pipeline.fit(data1)
dataset=pipelinefit.transform(data1)

In [23]:
data5=dataset.select("stars","filtered","ngrams")
data6=data5.withColumn("Features", mergeCols(col("filtered"), col("ngrams")))
data6.select("Features").collect()[0]

In [24]:
data6.dtypes

In [25]:
tokenizer2 = RegexTokenizer(inputCol="Features", outputCol="words")
cv2 = CountVectorizer(inputCol="words", outputCol="Count",minDF=4.0)
hashingTF2 = HashingTF(inputCol="words", outputCol="rawFeatures")
#featurizedData = hashingTF.transform(wordsData)
idf2 = IDF(inputCol="rawFeatures", outputCol="TFIDF")
pipeline2 = Pipeline(stages=[tokenizer2,cv2,hashingTF2,idf2])
pipelinefit2=pipeline2.fit(data6)
dataset2=pipelinefit2.transform(data6)

In [26]:
dataset.show(5)
dataset2.show(5)

In [27]:
dataset.groupBy("stars").count().show()

In [28]:
dataset_un_cv=dataset.selectExpr("Count as features","stars as label")
dataset_un_tfidf=dataset.selectExpr("TFIDF as features","stars as label")
dataset2_bi_cv=dataset2.selectExpr("Count as features","stars as label")
dataset2_bi_tfidf=dataset2.selectExpr("TFIDF as features","stars as label")

In [29]:
(trainingData11, testData11) = dataset_un_cv.randomSplit([0.75, 0.25], seed = 100)
(trainingData12, testData12) = dataset_un_tfidf.randomSplit([0.75, 0.25], seed = 100)
(trainingData21, testData21) = dataset2_bi_cv.randomSplit([0.75, 0.25], seed = 100)
(trainingData22, testData22) = dataset2_bi_tfidf.randomSplit([0.75, 0.25], seed = 100)

In [30]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=30, regParam=0.3, elasticNetParam=0.0)
lrModel11 = lr.fit(trainingData11)
lrModel12 = lr.fit(trainingData12)
lrModel21 = lr.fit(trainingData21)
lrModel22 = lr.fit(trainingData22)

In [31]:
predictions11 = lrModel11.transform(testData11)
predictions12 = lrModel12.transform(testData12)
predictions21 = lrModel21.transform(testData21)
predictions22 = lrModel22.transform(testData22)

In [32]:
predictions12.filter(predictions12['prediction'] == 1).select("features","label","probability","prediction").orderBy("probability", ascending=False).show(n = 10, truncate = 30)
#predictions12.show(5)

+------------------------------+-----+------------------------------+----------+
 features|label| probability|prediction|
+------------------------------+-----+------------------------------+----------+
(262144,[8630,35627,58211,7...| 1|[7.00099888300117E-6,0.2142...| 1.0|
(262144,[622,13963,17005,18...| 1|[6.989135517575573E-6,0.228...| 1.0|
(262144,[3067,7979,8607,863...| 4|[6.9858488874443265E-6,0.22...| 1.0|
(262144,[8527,24016,24113,3...| 2|[6.983870365137023E-6,0.225...| 1.0|
(262144,[7312,24113,46252,7...| 1|[6.974314429243404E-6,0.236...| 1.0|
(262144,[1886,2711,29238,32...| 1|[6.968822018509564E-6,0.236...| 1.0|
(262144,[2711,18270,24113,3...| 1|[6.9647806217050886E-6,0.23...| 1.0|
(262144,[329,12710,13781,13...| 1|[6.961165601019994E-6,0.224...| 1.0|
(262144,[7693,19398,24113,3...| 2|[6.958718684329267E-6,0.237...| 1.0|
(262144,[12320,12329,19164,...| 3|[6.956710868632208E-6,0.242...| 1.0|
+------------------------------+-----+------------------------------+----------+
only showing top 10 rows

In [33]:
#testing accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator11 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator11.evaluate(predictions11)

Out[ 31 ]: 0.49411175806307367

In [34]:
evaluator12 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator12.evaluate(predictions12)

Out[ 32 ]: 0.47573540395132113

In [35]:
evaluator21 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator21.evaluate(predictions21)

Out[ 33 ]: 0.5119723006324494

In [36]:
evaluator22 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator22.evaluate(predictions22)

Out[ 34 ]: 0.5014689789779911

In [37]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=2.0, modelType="multinomial")
# train the model
model11 = nb.fit(trainingData11)
model12 = nb.fit(trainingData12)
model21 = nb.fit(trainingData21)
model22 = nb.fit(trainingData22)
# select example rows to display.

In [38]:
testData11.show(20)
testData21.show(20)

+--------------------+-----+
 features|label|
+--------------------+-----+
 (99288,[],[])| 4|
 (99288,[],[])| 5|
 (99288,[],[])| 5|
(99288,[0,1,2,3,4...| 4|
(99288,[0,1,2,3,4...| 5|
(99288,[0,1,2,3,4...| 3|
(99288,[0,1,2,3,4...| 4|
(99288,[0,1,2,3,4...| 5|
(99288,[0,1,2,3,4...| 1|
(99288,[0,1,2,3,4...| 5|
(99288,[0,1,2,3,4...| 4|
(99288,[0,1,2,3,4...| 5|
(99288,[0,1,2,3,4...| 4|
(99288,[0,1,2,3,4...| 4|
(99288,[0,1,2,3,4...| 5|
(99288,[0,1,2,3,4...| 3|
(99288,[0,1,2,3,4...| 5|
(99288,[0,1,2,3,4...| 1|
(99288,[0,1,2,3,4...| 4|
(99288,[0,1,2,3,4...| 5|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
 features|label|
+--------------------+-----+
 (194857,[],[])| 1|
 (194857,[],[])| 3|
 (194857,[],[])| 4|
 (194857,[],[])| 4|
 (194857,[],[])| 5|
 (194857,[],[])| 5|
 (194857,[],[])| 5|
 (194857,[],[])| 5|
 (194857,[],[])| 5|
 (194857,[],[])| 5|
(194857,[0,1,2,3,...| 3|
(194857,[0,1,2,3,...| 3|
(194857,[0,1,2,3,...| 5|
(194857,[0,1,2,3,...| 1|
(194857,[0,1,2,3,...| 3|
(194857,[0,1,2,3,...| 4|
(194857,[0,1,2,3,...| 3|
(194857,[0,1,2,3,...| 4|
(194857,[0,1,2,3,...| 3|
(194857,[0,1,2,3,...| 4|
+--------------------+-----+
only showing top 20 rows

In [39]:
predictions_nb11 = model11.transform(testData11)
predictions_nb12 = model12.transform(testData12)
predictions_nb12.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|label| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
(262144,[6,2711,4...| 4|[-8383.9571040947...|[0.98136267189978...| 0.0|
(262144,[10,14,18...| 5|[-7552.3539950350...|[6.32691183433727...| 1.0|
(262144,[10,14,39...| 4|[-3547.9360584679...|[6.98207869704518...| 3.0|
(262144,[10,14,48...| 3|[-8554.6564685787...|[4.43598481915168...| 3.0|
(262144,[10,4470,...| 5|[-3633.8348344280...|[1.07519895465292...| 3.0|
(262144,[10,5381,...| 5|[-2818.2393983833...|[1.54617042193571...| 4.0|
(262144,[10,6113,...| 4|[-1608.4410516320...|[4.89184446376752...| 2.0|
(262144,[10,12710...| 5|[-1275.2389991056...|[2.30972143633591...| 4.0|
(262144,[13,14,30...| 4|[-7944.6182796292...|[3.81839664393343...| 3.0|
(262144,[13,14,30...| 4|[-8044.1337352272...|[2.47866475773427...| 3.0|
(262144,[14,29,32...| 4|[-4711.7728914082...|[5.24947254762324...| 2.0|
(262144,[14,34,18...| 3|[-3402.0281462855...|[4.33130096002828...| 2.0|
(262144,[14,61,16...| 4|[-6827.0700029787...|[4.19163225152563...| 2.0|
(262144,[14,67,84...| 5|[-10101.494776180...|[1.0,5.4129118008...| 0.0|
(262144,[14,67,17...| 4|[-4284.7159405885...|[1.16758266493876...| 3.0|
(262144,[14,67,53...| 4|[-2372.3590238833...|[1.60030361963835...| 3.0|
(262144,[14,67,60...| 3|[-3970.8152134079...|[7.24578626543443...| 3.0|
(262144,[14,67,84...| 5|[-2569.1711755203...|[1.46241496501796...| 4.0|
(262144,[14,78,19...| 4|[-5005.1818861654...|[9.68916240335920...| 3.0|
(262144,[14,78,32...| 4|[-5976.4590410372...|[8.69205290221318...| 3.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [40]:
predictions_nb21 = model21.transform(testData21)
predictions_nb22 = model22.transform(testData22)
predictions_nb22.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|label| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
(262144,[4,14,78,...| 3|[-20956.680376889...|[1.39017539924557...| 2.0|
(262144,[5,640,92...| 1|[-7856.9614372244...|[1.0,3.8798159839...| 0.0|
(262144,[5,1013,2...| 1|[-23392.753678364...|[1.0,1.9111383130...| 0.0|
(262144,[6,14,174...| 1|[-13141.252094528...|[1.0,1.1868243160...| 0.0|
(262144,[7,14,329...| 1|[-26237.224980771...|[1.0,1.4448307765...| 0.0|
(262144,[7,329,17...| 2|[-13103.408908627...|[1.0,2.0533997492...| 0.0|
(262144,[7,640,12...| 5|[-11712.237313427...|[1.58495580341446...| 4.0|
(262144,[7,1145,1...| 1|[-14777.440198054...|[1.60269539793856...| 1.0|
(262144,[8,14,329...| 1|[-33277.121967761...|[1.20812847763066...| 1.0|
(262144,[8,5173,1...| 4|[-2368.7375759203...|[5.47921754910174...| 2.0|
(262144,[10,528,2...| 5|[-12948.962069419...|[1.0,1.6412006602...| 0.0|
(262144,[10,943,1...| 4|[-4351.3229712952...|[4.97478338805065...| 3.0|
(262144,[10,4470,...| 5|[-10595.627967118...|[3.79345637268287...| 3.0|
(262144,[13,14,14...| 4|[-24039.955319850...|[5.81703268524282...| 3.0|
(262144,[13,14,24...| 3|[-11203.409593759...|[1.80188815452962...| 3.0|
(262144,[13,7897,...| 1|[-3066.8097044020...|[0.99999914779037...| 0.0|
(262144,[14,15,98...| 3|[-16860.480304057...|[0.0,6.6971809043...| 2.0|
(262144,[14,15,93...| 4|[-17063.378862390...|[5.80690789869550...| 4.0|
(262144,[14,19,32...| 3|[-63382.539694201...|[0.0,1.4961743005...| 2.0|
(262144,[14,35,23...| 4|[-12694.685769242...|[6.11585303812403...| 3.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [41]:
predictions_nb11_1=predictions_nb11.withColumn("label",col("label")-1)
predictions_nb12_1=predictions_nb12.withColumn("label",col("label")-1)
predictions_nb21_1=predictions_nb21.withColumn("label",col("label")-1)
predictions_nb22_1=predictions_nb22.withColumn("label",col("label")-1)

In [42]:
#test accuracy for naive bayes with unigram and count vectorizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2_11 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator2_11.evaluate(predictions_nb11_1)

Out[ 40 ]: 0.5425769514832036

In [43]:
#test for naive bayes with unigram and tfidf
evaluator2_12 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator2_12.evaluate(predictions_nb12_1)

Out[ 41 ]: 0.4862613160852701

In [44]:
#test for naive bayes with unigram+bigram and Count vectorizer
evaluator2_21 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator2_21.evaluate(predictions_nb21_1)

Out[ 42 ]: 0.5434527393702028

In [45]:
#test for naive bayes unigram+bigram with tfidf
evaluator2_22 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator2_22.evaluate(predictions_nb22_1)

Out[ 43 ]: 0.49524240330206726